In [1]:
#In this notebook, I'll be creating a simple dataset 
#that we can use to train up a toy model. 
#This dataset will be based on the match to a particular motif, CCATGT
#The score at each position in the genome will be based on how similar 
#it is to the motif.
MOTIF="CCATGT"
OTHER_MOTIF="TATAGT"
WORKING_DIRECTORY="/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo"
SRC_DIR="/n/projects/cm2363/chrombpnet-heavy/src"


In [2]:
for dirname in ["input", "data", "bed", "models", "shap", "json", "pred", "modisco"]:
    !mkdir -p {WORKING_DIRECTORY}/{DIRNAME}

In [3]:
import matplotlib.pyplot as plt
import pysam
import pyBigWig
import numpy as np
import json
import scipy.ndimage

In [4]:
GENOME_FASTA = "/n/data1/genomes/S_cerevisiae/sacCer3/all_chr.fa"
CHROM_SIZES = "/n/data1/genomes/indexes/sacCer3/extras/sacCer3.chrom.sizes"
genome = pysam.FastaFile(GENOME_FASTA)
bwHeader = []
for ref in genome.references:
    bwHeader.append((ref, genome.get_reference_length(ref)))

In [5]:
TEST_CHROMS = genome.references[:2]
VAL_CHROMS = genome.references[2:4]
TRAIN_CHROMS = genome.references[4:]

In [6]:
def getScore(motif, sequence):
    score = 1
    for i, c in enumerate(motif):
        if(sequence[i] == c):
            score *= 2
    return max(0.0, score-4.0)

In [7]:
with pyBigWig.open(WORKING_DIRECTORY + "/data/testData.bw", "w") as outBw:
    outBw.addHeader(bwHeader)
    for chrom in genome.references:
        print(chrom)
        vals = []
        otherVals = []
        chromSeq = genome.fetch(chrom, 0, genome.get_reference_length(chrom))
        for pos in range(genome.get_reference_length(chrom)-len(MOTIF)):
            seq = chromSeq[pos:pos+len(MOTIF)]
            scoreVal = getScore(MOTIF, seq)
            scoreOther = getScore(OTHER_MOTIF, seq)
            vals.append(scoreVal)
            otherVals.append(scoreOther)
        print("built")
        smoothVals = scipy.ndimage.gaussian_filter1d(otherVals, 15)*5
        smoothVals += scipy.ndimage.gaussian_filter1d(otherVals, 5)*2
        smoothVals += scipy.ndimage.gaussian_filter(vals, 1)
        smoothVals += np.array(vals)
        outBw.addEntries(chrom, 0, values=list(smoothVals), span=1, step=1)
    print("closing")
    outBw.close()

chrI
built
chrII
built
chrIII
built
chrIV
built
chrIX
built
chrM
built
chrV
built
chrVI
built
chrVII
built
chrVIII
built
chrX
built
chrXI
built
chrXII
built
chrXIII
built
chrXIV
built
chrXV
built
chrXVI
built
closing


In [ ]:
#So this bigwig will have two components to it - one will be a low-frequency 
#response to MOTIF_OTHER, and the second will be a very local response to MOTIF.
#I sure hope a neural network can learn something so simple! 
#In order to train, I need to generate training regions. 

In [8]:
OUTPUT_LENGTH=1000
N_LAYERS=6
CONV1_SIZE=7
PCONV_SIZE=25
input_length_str = !{SRC_DIR}/lengthCalc.py --output-len {OUTPUT_LENGTH} --n-dil-layers {N_LAYERS} --conv1-kernel-size {CONV1_SIZE} --profile-kernel-size {PCONV_SIZE} 

In [9]:
INPUT_LENGTH=int(input_length_str[0])
print(INPUT_LENGTH)
RECEPTIVE_FIELD=INPUT_LENGTH - OUTPUT_LENGTH+1
print(RECEPTIVE_FIELD)
MAX_JITTER = 100

1282
283


In [10]:
import pybedtools
def generateTilingRegions(genome, width, chromEdgeBoundary, spaceBetween, allowChroms):
    chromRegions = []
    numRegions = 0
    #To use window_maker from pybedtools, I first need to create a bed containing the chromosomes where I want regions made. 
    for chrom in genome.references:
        if(chrom not in allowChroms):
            continue
         
        startPos = chromEdgeBoundary
        chromSize = genome.get_reference_length(chrom)
        stopPos = chromSize - chromEdgeBoundary
        chromRegions.append(pybedtools.Interval(chrom, startPos, stopPos))
     
    windows = pybedtools.BedTool(chromRegions).window_maker(w=width, 
                                                            s=spaceBetween + width, 
                                                            g=CHROM_SIZES)
    return windows

with pysam.FastaFile(GENOME_FASTA) as genomeFp:
    w = generateTilingRegions(genomeFp, OUTPUT_LENGTH, 10000, 5000, TEST_CHROMS + TRAIN_CHROMS + VAL_CHROMS)
    w.saveas(WORKING_DIRECTORY+ "/bed/tiling_all.bed")

In [11]:
biasBwSpec = [{"file-name" : WORKING_DIRECTORY+"/data/testData.bw", "max-quantile" : 1, "min-quantile" : 0.01}]
prepareBedNonPeaksConfig = {
    "bigwigs" : biasBwSpec, 
    "splits" : {"test-chroms"  : TEST_CHROMS, 
                "val-chroms"   : VAL_CHROMS,
                "train-chroms" : TRAIN_CHROMS,
                "regions" : [WORKING_DIRECTORY + "/bed/tiling_all.bed"]},
    "genome" : GENOME_FASTA,
    "output-length" : OUTPUT_LENGTH,
    "input-length" : INPUT_LENGTH,
    "max-jitter" : MAX_JITTER,
    "output-prefix" : WORKING_DIRECTORY + "/bed/peak", 
    "resize-mode" : "center", 
    "verbosity" : "INFO"}

with open(WORKING_DIRECTORY + "/json/prepareBedPeaks.json", "w") as fp:
    json.dump(prepareBedNonPeaksConfig, fp, indent=4)
    print(json.dumps(prepareBedNonPeaksConfig, indent=4))

{
    "bigwigs": [
        {
            "file-name": "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/data/testData.bw",
            "max-quantile": 1,
            "min-quantile": 0.01
        }
    ],
    "splits": {
        "test-chroms": [
            "chrI",
            "chrII"
        ],
        "val-chroms": [
            "chrIII",
            "chrIV"
        ],
        "train-chroms": [
            "chrIX",
            "chrM",
            "chrV",
            "chrVI",
            "chrVII",
            "chrVIII",
            "chrX",
            "chrXI",
            "chrXII",
            "chrXIII",
            "chrXIV",
            "chrXV",
            "chrXVI"
        ],
        "regions": [
            "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/bed/tiling_all.bed"
        ]
    },
    "genome": "/n/data1/genomes/S_cerevisiae/sacCer3/all_chr.fa",
    "write-counts-to": "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/bed/nonpeak_all.stats",
    "output-length": 

In [12]:
!{SRC_DIR}/prepareBed.py {WORKING_DIRECTORY}/json/prepareBedPeaks.json

INFO:root:Starting bed file generation.
INFO:root:Training regions: 1552
INFO:root:Validation regions: 309
INFO:root:Testing regions: 175
INFO:root:Rejected on loading: 0
INFO:root:Validating training regions.
100%|█████████████████████████████████████| 1534/1534 [00:01<00:00, 1521.31it/s]
INFO:root:Total surviving regions: 1518
INFO:root:Validating validation regions.
100%|███████████████████████████████████████| 307/307 [00:00<00:00, 1530.49it/s]
INFO:root:Total surviving regions: 303
INFO:root:Validating testing regions.
100%|███████████████████████████████████████| 173/173 [00:00<00:00, 1489.92it/s]
INFO:root:Total surviving regions: 171
INFO:root:Regions saved.


In [18]:
for split in ["train", "val"]:
    heads = [{"bigwig-files" : [WORKING_DIRECTORY+"/data/testData.bw"]}]
    config = {"genome" : GENOME_FASTA, 
              "input-length" : INPUT_LENGTH,
              "output-length" : OUTPUT_LENGTH,
              "max-jitter" : MAX_JITTER,
              "regions" : WORKING_DIRECTORY + "/bed/peak_" + split + ".bed",
              "output-h5" : WORKING_DIRECTORY + "/input/" + split + ".h5",
              "heads" : heads,
              "verbosity" : "DEBUG"}
    configFname =WORKING_DIRECTORY + "/json/prepareInput_" + split+ ".json" 
    with open(configFname, "w") as fp:
        json.dump(config, fp)
    !{SRC_DIR}/prepareTrainingData.py {configFname}

DEBUG:root:Sequence dataset created.
DEBUG:root:Added data for head 0
DEBUG:root:Sequence dataset created.
DEBUG:root:Added data for head 0


In [19]:
heads = [{"num-tasks" : 1, 
          "profile-loss-weight" : 1, 
          "head-name" : "solo",
          "counts-loss-weight" : 10}]
#print(heads)

#And now the whole config file:
soloTrainConfig = {
    "settings" : {
        "output-prefix" : WORKING_DIRECTORY + "/models/solo", 
        "epochs" : 20,
        "early-stopping-patience" : 20,
        "batch-size" : 128,
        "learning-rate" : 0.004,
        "learning-rate-plateau-patience" : 5,
        "max-jitter" : 100,
        "architecture" : {
            "architecture-name" : "bpnet", 
            "input-length" : INPUT_LENGTH,
            "output-length" : OUTPUT_LENGTH,
            "model-name" : "solo",
            "model-args" : "",
            "filters" : 16,
            "layers" : N_LAYERS,
            "input-filter-width" : CONV1_SIZE,
            "output-filter-width" : PCONV_SIZE
        }
    },
    "train-data" : WORKING_DIRECTORY + "/input/train.h5",
    "val-data" : WORKING_DIRECTORY + "/input/val.h5",
    "heads" : heads,
    "verbosity" : "WARNING"
}

print(json.dumps(soloTrainConfig, indent=4))

with open(WORKING_DIRECTORY + "/json/trainSolo.json", "w") as fp:
    json.dump(soloTrainConfig, fp)

{
    "settings": {
        "output-prefix": "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/models/solo",
        "epochs": 20,
        "early-stopping-patience": 20,
        "batch-size": 128,
        "learning-rate": 0.004,
        "learning-rate-plateau-patience": 5,
        "max-jitter": 100,
        "architecture": {
            "architecture-name": "bpnet",
            "input-length": 1282,
            "output-length": 1000,
            "model-name": "solo",
            "model-args": "",
            "filters": 16,
            "layers": 6,
            "input-filter-width": 7,
            "output-filter-width": 25
        }
    },
    "train-data": "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/input/train.h5",
    "val-data": "/n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/input/val.h5",
    "heads": [
        {
            "num-tasks": 1,
            "profile-loss-weight": 1,
            "head-name": "solo",
            "counts-loss-weight": 10
        }
    ],
 

In [20]:
!{SRC_DIR}/trainSoloModel.py {WORKING_DIRECTORY}/json/trainSolo.json

Epoch 1/20
11/12 [==========================>...] - ETA: 0s - loss: 3859.2468 - solo_profile_solo_loss: 3842.3464 - solo_logcounts_solo_loss: 1.6900
Epoch 1: val_loss improved from inf to 2872.70142, saving model to /n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/models/solo.checkpoint.model
2022-12-07 15:42:31.576694: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
12/12 [==============================] - 11s 462ms/step - loss: 3778.9407 - solo_profile_solo_loss: 3763.2898 - solo_logcounts_solo_loss: 1.5651 - val_loss: 2872.7014 - val_solo_profile_solo_loss: 2872.4390 - val_solo_logcounts_solo_loss: 0.0262 - lr: 0.0040
Epoch 2/20
 9/12 [=====================>........] - ETA: 0s - loss: 2792.6311 - solo_profile_solo_loss: 2791.4478 - solo_logcounts_solo_loss: 0.1184
Epoch 2: val_loss improved from 2872.70142 to 2664.73999, saving model to /n/projects/cm2363/chrombpnet-heavy/test/s

In [23]:
predictConfig = {
    "settings" : {
        "genome" : GENOME_FASTA, 
        "output-h5" : WORKING_DIRECTORY + "/pred/solo.h5", 
        "batch-size" : 128,
        "heads" : 1,
        
        "architecture" : {
            "model-file" : WORKING_DIRECTORY + "/models/solo.model",
            "input-length" : INPUT_LENGTH,
            "output-length" : OUTPUT_LENGTH
        }
    },
    "bed-file" : WORKING_DIRECTORY + "/bed/peak_all.bed",
    "verbosity" : "DEBUG"
}


with open(WORKING_DIRECTORY + "/json/predict.json", "w") as fp:
    json.dump(predictConfig, fp)

In [24]:
!{SRC_DIR}/makePredictionsBed.py {WORKING_DIRECTORY}/json/predict.json

INFO:root:GPU memory growth enabled.
DEBUG:root:Opening output hdf5 file.
INFO:root:Loading regions
INFO:root:Input prepared. Loading model.
INFO:root:Model loaded. Predicting.
16/16 [==============================] - 5s 21ms/step
INFO:root:Predictions complete. Writing hdf5.
INFO:root:Writing predictions
DEBUG:h5py._conv:Creating converter from 5 to 3
INFO:root:Datasets created. Populating regions.
INFO:root:Writing predictions.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 116.36it/s]
INFO:root:File saved.


In [25]:
!{SRC_DIR}/predictToBigwig.py \
    --h5 {WORKING_DIRECTORY}/pred/solo.h5 \
    --bw {WORKING_DIRECTORY}/pred/solo.bw \
    --head-id 0 \
    --task-id 0 \
    --mode profile \
    --verbose


INFO:root:Starting to write /n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/pred/solo.bw, head 0 task 0
INFO:root:Added header.
INFO:root:Loading coordinate data
INFO:root:Region data loaded. Sorting.
100%|██████████████████████████████████| 1992/1992 [00:00<00:00, 1005421.61it/s]
INFO:root:Generated list of regions to sort.
INFO:root:Region order calculated.
INFO:root:Loading head data.
INFO:root:Starting to write data.
100%|█████████████████████████████████████| 1992/1992 [00:01<00:00, 1578.03it/s]
INFO:root:Closing bigwig.
INFO:root:Bigwig closed.


In [26]:
!{SRC_DIR}/metrics.py \
    --reference {WORKING_DIRECTORY}/data/testData.bw \
    --pred {WORKING_DIRECTORY}/pred/solo.bw \
    --regions {WORKING_DIRECTORY}/bed/peak_all.bed \
    --threads 70 \
    --apply-abs

reference /n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/data/testData.bw predicted /n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/pred/solo.bw regions /n/projects/cm2363/chrombpnet-heavy/test/simpleDemo/bed/peak_all.bed
100%|█████████████████████████████████████| 1992/1992 [00:00<00:00, 5123.97it/s]
/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/lib/python3.10/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
metric    	      0.000000%	     25.000000%	     50.000000%	     75.000000%	    100.000000%	regions
mnll      	            nan	            nan	            nan	            nan	            nan	1992
jsd       	       0.021363	       0.027448	       0.030071	       0.033441	       0.063595	1992
pearsonr  	       0.964279	       0.979569	       0.981933	       0.983847	       0.990021	1992
spearmanr 	       0.963865	       0.981745	       0.984461	       0.986796	       0.993471	1992
Counts p

In [ ]:
#Now it's time to generate importance scores. 

In [27]:
interpretConfig =  {
        "genome" : GENOME_FASTA,
        "bed-file" : WORKING_DIRECTORY + "/bed/peak_all.bed",
        "model-file" : WORKING_DIRECTORY + "/models/solo.model", 
        "input-length" : INPUT_LENGTH,
        "output-length" : OUTPUT_LENGTH,
        "heads" : 1,
        "head-id": 0,
        "profile-task-ids" : [0],
        "profile-h5" : WORKING_DIRECTORY + "/shap/profile.h5",
        "counts-h5" : WORKING_DIRECTORY + "/shap/counts.h5",
        "num-shuffles" : 20,
        "verbosity" : "DEBUG"}

with open(WORKING_DIRECTORY + "/json/shap.json", "w") as fp:
    json.dump(interpretConfig, fp)

In [28]:
!{SRC_DIR}/interpretFlat.py {WORKING_DIRECTORY}/json/shap.json

INFO:root:GPU memory growth enabled.


INFO:root:AddV2used in model but handling of op is not specified by shap; will use original  gradients
INFO:root:StopGradientused in model but handling of op is not specified by shap; will use original  gradients
INFO:root:SpaceToBatchNDused in model but handling of op is not specified by shap; will use original  gradients
INFO:root:BatchToSpaceNDused in model but handling of op is not specified by shap; will use original  gradients
  0%|                                                  | 0/1992 [00:00<?, ?it/s]2022-12-08 10:42:17.729125: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.66GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-12-08 10:42:17.735176: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocat

In [29]:
!{SRC_DIR}/shapToBigwig.py \
    --h5 {WORKING_DIRECTORY}/shap/profile.h5 \
    --bw {WORKING_DIRECTORY}/shap/profile.bw \
    --verbose

INFO:root:Bigwig header[('chrI', 230218), ('chrII', 813184), ('chrIII', 316620), ('chrIV', 1531933), ('chrIX', 439888), ('chrM', 85779), ('chrV', 576874), ('chrVI', 270161), ('chrVII', 1090940), ('chrVIII', 562643), ('chrX', 745751), ('chrXI', 666816), ('chrXII', 1078177), ('chrXIII', 924431), ('chrXIV', 784333), ('chrXV', 1091291), ('chrXVI', 948066)]
INFO:root:Files opened; writing regions
100%|██████████████████████████████████████| 1992/1992 [00:03<00:00, 654.35it/s]
INFO:root:Regions written; closing bigwig.
INFO:root:Done saving shap scores.


In [30]:
#To run modisco, the scores have to be saved as a numpy array. 
!{SRC_DIR}/shapToNumpy.py \
    --h5 {WORKING_DIRECTORY}/shap/profile.h5 \
    --seqs {WORKING_DIRECTORY}/shap/seq.npy \
    --scores {WORKING_DIRECTORY}/shap/scores.npy

(1992, 4, 1282)
(1992, 4, 1282)


In [34]:
!modisco motifs --sequences {WORKING_DIRECTORY}/shap/seq.npy \
    --attributions {WORKING_DIRECTORY}/shap/scores.npy \
    --max_seqlets 2000 \
    --output {WORKING_DIRECTORY}/modisco/profile.h5 \
    --verbose

Using 2000 positive seqlets
^C
Traceback (most recent call last):
  File "/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/bin/modisco", line 103, in <module>
    pos_patterns, neg_patterns = modiscolite.tfmodisco.TFMoDISco(
  File "/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/lib/python3.10/site-packages/modiscolite/tfmodisco.py", line 310, in TFMoDISco
    pos_patterns = seqlets_to_patterns(seqlets=pos_seqlets,
  File "/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/lib/python3.10/site-packages/modiscolite/tfmodisco.py", line 215, in seqlets_to_patterns
    patterns = _patterns_from_clusters(filtered_seqlets, 
  File "/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/lib/python3.10/site-packages/modiscolite/tfmodisco.py", line 106, in _patterns_from_clusters
    pattern = aggregator.merge_in_seqlets_filledges(
  File "/n/projects/cm2363/utils/src/anaconda3/envs/bpreveal/lib/python3.10/site-packages/modiscolite/aggregator.py", line 100, in merge_in_seqlets_filledge

In [37]:
!modisco report \
    --h5py {WORKING_DIRECTORY}/modisco/profile.h5 \
    --output {WORKING_DIRECTORY}/modisco/profile/ \
    --suffix profile/ \
    --meme_db /n/data1/JASPAR/2020/JASPAR2020_CORE_vertebrates_redundant_pfms.meme